This script is used to create the geodatabase and features needed to analyze the networks and eventually perform the operations necesarry to create a relationship between the networks. The first cell imports the required modules and defines the spatial reference that will be used when createing the feature classes. party party party 

In [ ]:
import os
import arcpy as ap

ap.SpatialReference(102382)
spatial_reference = ap.SpatialReference(102382)
ap.env.XYDomain = "-180 -90 180 90"

The shapefiles to be analyzed are to be stored in the network shapefiles directory. If downloaded as a repository from GitHub, or an Anaconda project this directory is located in the same directory as this folder. Finally, this cell creates a list of the shapefiles.

In [ ]:
local_shapefiles_directory = os.getcwd() + '\\network_shapefiles'
local_geodatabase = os.getcwd() + '\\working_files.gdb'

if ap.Exists(local_geodatabase) == False:
    ap.CreateFileGDB_management(os.getcwd(), 'working_files.gdb')

ap.env.workspace = local_shapefiles_directory

network_shapefiles = ap.ListFeatureClasses()

A number of feature classes are required to complete this analysis. First, it's necesarry todetermine where all the actual intersections (and overpasses) are located. THis is accomplished by dissolving the entire network and then performing a feature to line conversion on the dissolved layers, the endpoints of these lines will be used in the next step to define crossings. 

In [ ]:
ap.env.workspace = local_geodatabase

for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '//' + each_network).baseName

    if ap.Exists(network_name) == False:
        nearby_network_links(local_shapefiles_directory)
        ap.FeatureClassToFeatureClass_conversion('%s_lyr' % network_name,
                                                 local_geodatabase, 
                                                 '%s' % network_name,
                                                 '',
                                                 '',
                                                 '')
    
    if ap.Exists('%s_dissolved_to_line' % network_name) == False:
        
        ap.Dissolve_management('%s_lyr' % network_name, 
                                   'in_memory/%s_dissolved'% network_name,
                                   '',
                                   '',
                                   'MULTI_PART',
                                   'DISSOLVE_LINES')
                
        ap.FeatureToLine_management('in_memory/%s_dissolved' % network_name,
                                            '%s_dissolved_to_line' % network_name)
        
        ap.Delete_management('in_memory/%s_dissolved' % network_name)
        
    if ap.Exists('%s_crossings' % network_name) == False:
        ap.CreateFeatureclass_management(local_geodatabase, '%s_crossings' % network_name,
                                     'POINT',
                                     "",
                                     "",
                                     "",
                                     spatial_reference)
    else:
        ap.DeleteRows_management('%s_crossings' % network_name)
        pass
    
    add_fields = ['crossings',
                  'link_ids',
                  'intersections',
                  'overpasses',
                  ]

    network_abbrev = network_name[:3]

    for field in add_fields:
        try:
            ap.AddField_management('%s/%s_crossings' % (local_geodatabase, network_name),
                                       '%s_%s' % (network_abbrev, field),
                                       'STRING',
                                       '',
                                       '',
                                       '255')
        except:
            pass